# Introduction

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/teseoch/fem-intro/master?filepath=fem-intro.ipynb)
The notebook can be interactively run in binder!

Let $\Omega$ be the domain, we aim at solving the Laplace equation:
$$\Delta u =0$$

Subject to boundary conditions $g$
$$u|_{\partial\Omega} = g,$$

where $u|_{\partial\Omega}$ is the boundary of $\Omega$

# Weak Form

Instread of solving
$$\Delta u =0$$

we multiply it by a **test funciton** $v$ and integrate 

$$\int_\Omega\Delta u v =0, \qquad \forall v$$

This equation is called **weak form** of the original PDE, and if it hold for any $v$ then $u$ is also a solution of the original PDE (**strong form**)

We can now use integration by parts:
$$\int_\Omega\Delta u v = \int_\Omega\nabla u \cdot \nabla v = 0, \qquad \forall v$$

# Discretization

We express the unknown function $u$ in term of a **dicrete** basis $\phi_i$, $i=0\dots,n$, 

that is
$$u=\sum_{i=0}^n u_i \phi_i$$

We use the same definition for $v$ and plug it in the weak form
$$\sum_{i=0}^n u_i \int_\Omega\nabla \phi_i \cdot \nabla \phi_j = 0, \qquad \forall j=0,\dots,n$$

This expression can be rewritten in matrix form
$$
L u  =0,
$$
where
$$
L_{i,j} = \int_\Omega\nabla \phi_i \cdot \nabla \phi_j
$$
and $u$ is the vector containing the $u_i$.

# 1D Example

In [1]:
import numpy as np
import scipy.sparse as spr
from scipy.sparse.linalg import spsolve

import plotly.offline as plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff

#Necessary for the notebook
plotly.init_notebook_mode(connected=True)

The domain $\Omega = [0, 1]$, which we discretize with $n$ segments (or elements) $s_i$

In [58]:
#domain
omega = np.array([0, 1])

#number of bases and elements
n_elements = 10
n_bases = n_elements + 1

#segments
# s = np.linspace(omega[0], omega[1], num=n_elements+1)
s = np.cumsum(np.random.rand(n_elements+1))
s = (s-s[0])/(s[-1]-s[0])


#plot
fig = go.Figure(data=[go.Scatter(x=s, y=np.zeros(s.shape), mode='lines+markers')])
plotly.iplot(fig)

In [41]:
phis = []

for i in range(n_bases):
    phi = np.zeros(s.shape)
    phi[i] = 1
    phis.append(go.Scatter(x=s, y=phi, mode='lines+markers', name="$\phi_{{{}}}$".format(i)))

fig = go.Figure(data=phis)
plotly.iplot(fig)

# Local bases

For simplicity we define the **reference element** $\hat s= [0, 1]$, a segment of unit length

on each element we have only 2 **non-zero** local bases. We define thier "piece" on $\hat s$

In [42]:
#definition of bases
def hat_phi0(x):
    return 1-x
def hat_phi1(x):
    return x

We can now plot the two bases

In [43]:
x = np.linspace(0, 1)
fig = go.Figure(data=[
    go.Scatter(x=x, y=hat_phi0(x), mode='lines', name="$\hat\phi_0$"),
    go.Scatter(x=x, y=hat_phi1(x), mode='lines', name="$\hat\phi_1$")
])
plotly.iplot(fig)

Because of the reference element we define the **geometric mappings** that maps the local segment $\hat s$ to each global ones $s_i$:
$$g_i(x) = s_{i,0} + x (s_{i,1} - s_{i,0})$$

where $s_{i,0}$ and $s_{i,1}$ are the start and end point of $s_i$.

This localization forces to keep track of the mapping between the 2 local nodes and their respective global indices, this mapping is called **local to global**.

We now can further rearrange the weak form in term of element integrals

$$
\sum_{i=0}^n u_i \int_\Omega\nabla \phi_i \cdot \nabla \phi_j =
\sum_{e=0}^{n_{el}}\sum_{i=0}^n u_i \int_{s_e}\nabla \phi_i \cdot \nabla \phi_j = 0, \qquad \forall j=0,\dots,n$$

we perform a change of variable in the integral and the **jacobian** of the geometric mapping appers because of chain rule and change of variable.

Note that most of the terms are zero since only 2 bases are not zero, and we itroduce the **local to global** mapping $g_e^i$ that maps the local index $i=0,1$ of element $e$ to its corresponding global one.

$$
\sum_{e=0}^{n_{el}}\sum_{i=0}^1 u_{g_e^i} \int_{\hat s_j} \frac{\nabla \hat\phi_i}{s_{j, 1} - s_{j, 0}} \cdot \frac{\nabla \hat\phi_j}{s_{j, 1} - s_{j, 0}} (s_{j, 1} - s_{j, 0})=
%
\sum_{e=0}^{n_{el}}\sum_{i=0}^1 u_{g_e^i} \int_{\hat s_j} \frac{\nabla \hat\phi_i \cdot \nabla \hat\phi_j}{s_{j, 1} - s_{j, 0}} 
, \qquad \forall j=0,1$$


Note that we now need the gradients of the local bases

```python
def hat_phi0(x):
    return 1-x
def hat_phi1(x):
    return x
```

In [44]:
def grad_hat_phi0(x):
    return -np.ones(x.shape)
def grad_hat_phi1(x):
    return np.ones(x.shape)

# Basis construction

In [45]:
elements = []
for e in range(n_elements):
    el = {}
    
    el["n_bases"] = 2
    
    #2 bases
    el["phi"] = [hat_phi0, hat_phi1]
    el["grad_phi"] = [grad_hat_phi0, grad_hat_phi1]
    
    #local to global mapping
    el["loc_2_glob"] = [e, e+1]
    
    #geometric mapping
    el["gmapping"] = lambda x, e=e : s[e] + x*(s[e+1]-s[e])
    el["grad_gmapping"] = lambda x : (s[e+1]-s[e])
    
    elements.append(el)

We define a function to interpolate the $u_i$ using the local to global, geometric mapping, and local bases to interpolate the data

In [46]:
def interpolate(ui):
    u = np.array([])
    x = np.array([])

    xhat = np.linspace(0, 1)


    for e in range(n_elements):
        el = elements[e]
    
        uloc = np.zeros(xhat.shape)

        for i in range(el["n_bases"]):
            glob_node = el["loc_2_glob"][i]
            loc_base = el["phi"][i]
        
            uloc += ui[glob_node] * loc_base(xhat)
    
        u = np.append(u, uloc)
        x = np.append(x, el["gmapping"](xhat))
    
    return x, u

We can generate a random vector $ui$ and use the previous function

In [47]:
ui = np.random.rand(n_bases)

x, u = interpolate(ui)


fig = go.Figure(data=[
    go.Scatter(x=x, y=u, mode='lines'),
    go.Scatter(x=s, y=ui, mode='markers'),
])
plotly.iplot(fig)

# Assembly

We are now ready the assemble the global stiffness matrix. Note that the integrals are performed with `quadpy`

In [48]:
import quadpy

scheme = quadpy.line_segment.gauss_patterson(5)


rows = []
cols = []
vals = []



for e in range(n_elements):
    el = elements[e]

    for i in range(el["n_bases"]):
        for j in range(el["n_bases"]):
            val = scheme.integrate(
                lambda x:
                el["grad_phi"][i](x) * el["grad_phi"][j](x) / el["grad_gmapping"](x),
                [0.0, 1.0])
            
            rows.append(el["loc_2_glob"][i])
            cols.append(el["loc_2_glob"][j])
            vals.append(val)

            
rows = np.array(rows)
cols = np.array(cols)
vals = np.array(vals)

L = spr.coo_matrix((vals, (rows, cols)))
L = spr.csr_matrix(L)

In [49]:
L.toarray()

array([[ 5.31427638e+00, -5.31427638e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-5.31427638e+00,  1.00000531e+06, -1.00000000e+06,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.00000000e+06,  1.00000458e+06,
        -4.58164536e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.58164536e+00,
         1.70829273e+01, -1.25012820e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.25012820e+01,  3.43005211e+01, -2.17992391e+01,
  

We set the row zero and `n_elements` to identity for the boundary conditions

In [50]:
for bc in [0, n_elements]:
    _, nnz = L[bc,:].nonzero()
    for j in nnz:
        if j != bc:
            L[bc, j] = 0.0
    L[bc, bc] = 1.0

In [51]:
L.A

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-5.31427638e+00,  1.00000531e+06, -1.00000000e+06,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.00000000e+06,  1.00000458e+06,
        -4.58164536e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.58164536e+00,
         1.70829273e+01, -1.25012820e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.25012820e+01,  3.43005211e+01, -2.17992391e+01,
  

We set the righ-hand side to zero, and set the two boundary condition to be 1 and 4

In [52]:
f = np.zeros((n_bases, 1))
f[0] = 1
f[-1] = 4

We now solve $Lui=0$ for $ui$

In [53]:
ui = spsolve(L, f)

We now plot, we expect a line!

In [54]:
x, u = interpolate(ui)


fig = go.Figure(data=[
    go.Scatter(x=x, y=u, mode='lines', name="solution"),
    go.Scatter(x=s, y=ui, mode='markers', name="$ui$"),
])
plotly.iplot(fig)

# Mass Matrix

We change the pde from the Laplce to Poisson
$$
-\Delta u = f
$$

If we assume that $f$ is also expressed in terms of $\phi_i$ we can rewrite the weak form as

$$\sum_{i=0}^n u_i \int_\Omega\nabla \phi_i \cdot \nabla \phi_j = \sum_{i=0}^n f_i \int_\Omega\phi_i \phi_j, \qquad \forall j=0,\dots,n$$
which can be represented in matrix form
$$
L u = M f,
$$
where $f$ is the vector of $f_i$ and
$$
M_{i,j} = \int_\Omega\phi_i \phi_j
$$
is the **mass matrix**.

As for the stiffness matrix it can be localized
$$
M_{i,j} = \int_{\hat s_j} \hat\phi_i \cdot \hat\phi_j \,(s_{j, 1} - s_{j, 0}) 
$$
and $(s_{j, 1} - s_{j, 0})$ appears because of the change of variable 

In [55]:
import quadpy

scheme = quadpy.line_segment.gauss_patterson(5)


rows = []
cols = []
vals = []



for e in range(n_elements):
    el = elements[e]

    for i in range(el["n_bases"]):
        for j in range(el["n_bases"]):
            val = scheme.integrate(
                lambda x:
                el["phi"][i](x) * el["phi"][j](x) * el["grad_gmapping"](x),
                [0.0, 1.0])
            
            rows.append(el["loc_2_glob"][i])
            cols.append(el["loc_2_glob"][j])
            vals.append(val)

            
rows = np.array(rows)
cols = np.array(cols)
vals = np.array(vals)

M = spr.coo_matrix((vals, (rows, cols)))
M = spr.csr_matrix(M)

Now we set $f=4$ and zero boundary conditions

In [56]:
f = 4*np.ones((n_bases, 1))
f[0] = 0
f[-1] = 0

f = M*f

We now solve $Lui=f$ for $ui$

In [57]:
ui = spsolve(L, f)

x, u = interpolate(ui)


fig = go.Figure(data=[
    go.Scatter(x=x, y=u, mode='lines', name="solution"),
    go.Scatter(x=s, y=ui, mode='markers', name="$ui$"),
])
plotly.iplot(fig)